In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import os
import glob
from os import walk
from matplotlib.gridspec import GridSpec
from IPython.display import display, clear_output
from nano_lab import experiments,get_data
from mpl_toolkits.axes_grid1 import make_axes_locatable,ImageGrid
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import pandas as pd
mpl.rcParams.update(mpl.rcParamsDefault)
# plt.style.use('../src/plotstyle.mplstyle')
# exp= experiments('nano-lab','nsom','fg163r',True)
exp=get_data('nano-lab','nsom','fg163r',True)
meas = exp.exp_meas
if meas:
    attrs, data = exp.get_spectra(meas[0])
else :
    print('Error')
    
afm=data[0][:,:,0]

In [3]:
def get_data_spectra(exp,sel_meas):
    meas = exp.exp_meas
    if meas:
        attrs, data = exp.get_spectra(sel_meas)
        if data[0].ndim>2:
            afm  = data[0][:,:,0]
            nsom = data[0][:,:,0]
            return afm, nsom, attrs
        else:
            afm = data[0]
            return afm, attrs

data = get_data_spectra(exp,'2022-10-28-nsom.h5')    
    
dframe = pd.DataFrame.from_dict([attrs]).T
dframe.columns=["Value"]
dframe.columns.names=['Values']
dframe

Values,Value
Inicio X,0.0
Inicio Y,0.0
Tipo,R
Unidades,nm
fin X,4000.0
fin Y,4000.0
paso,250.0


In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def zmap(exptype):
    z = exptype.T
    nx, ny = z.shape[1], z.shape[0]
    x, y = np.meshgrid(np.arange(0, nx, 1), np.arange(0, ny, 1))
    xx = np.arange(0, nx, 1)
    yy = np.arange(0, ny, 1)
    offs = np.min(z)*0.13
    if np.min(z)-offs > 1.e-4:
        z_offset=(np.min(z)-offs)*np.ones(z.shape)#
    else:
        z_offset=(np.min(z)+offs)*np.ones(z.shape)#

    x_offset=-np.min(xx)*np.ones(z.shape)
    y_offset=np.min(yy)*np.ones(z.shape)
    proj_z=lambda x, y, z: z#projection in the z-direction
    colorsurfz=proj_z(x,y,z)
    proj_x=lambda x, y, z: x
    colorsurfx=proj_z(x,y,z)
    proj_y=lambda x, y, z: y
    colorsurfy=proj_z(x,y,z)

    class Results(): pass
    results = Results()
    results.colorsurfz=  colorsurfz
    results.z_offset = z_offset
    results.x = x
    results.y = y
    return results

def fig_3d_2d_layout(data,template,attrs,pixel, color='Plotly3',**kwargs):
    
    if  'tick_step' in kwargs:
        tick_step = kwargs.pop('tick_step')
    else:
        tick_step = 5
        
    z = data
    nx = data.shape[0]
    ny = data.shape[1]
    xm,ym = np.meshgrid(np.arange(0, nx, 1), np.arange(0, ny, 1))
    x = np.arange(0, nx, 1)
    y = np.arange(0, ny, 1)
    expsoffsets = zmap(z)
    
    if attrs:
        xi =  int(attrs['Inicio X'])
        xf =  int(round(attrs['fin X ']/attrs['paso']))
        print(xf)
        yi =  int(attrs['Inicio Y'])
        yf =  int(round(attrs['fin Y ']/attrs['paso']))    
        step = attrs['paso']
        xt = np.arange(xi,xf,tick_step)
        yt = np.arange(yi,yf,tick_step)
        xtl = xt*step
        ytl = yt*step
    else:
        xi =  x[0]
        xf =  x[-1]
        yi =  y[0]
        yf =  y[-1]
        step = 1
        xt = np.arange(xi,xf,tick_step)
        yt = np.arange(yi,yf,tick_step)
        xtl = xt*step
        ytl = yt*step
        # utils.error_alert()
        
        
    
    fig = make_subplots(
            rows=2, cols=2, 
            column_widths=[0.6, 0.4],
            row_heights=[0.5, 0.5],
            specs=[[{'type': 'surface','rowspan':2},{'type': 'contour'}],
                    [None, {'type':'scatter'}]],
            horizontal_spacing=0.00,
            vertical_spacing=0.05,
            shared_xaxes=True,
            
            )


        # Generate data
    fig.add_trace(go.Surface(z=data.T,showscale=False,colorscale=color),row=1, col=1)
    fig.add_trace(go.Surface(z=list(expsoffsets.z_offset),  # type: ignore
                x=list(expsoffsets.x),
                y=list(expsoffsets.y),
                colorscale=color,
                showscale=False, 
                 surfacecolor=expsoffsets.colorsurfz),row=1, col=1)
    
    
    
    fig.add_trace(go.Heatmap(z=data.T,connectgaps=True, zsmooth='best',colorscale=color),row=1, col=2)

    pixel_line_profile = go.Scatter(x=x,y=z[:,pixel],mode='lines+markers',marker_line_width=2,marker_size=10,)
    pixel_line = go.Scatter(x=[xi,xf],y=[pixel,pixel],marker_line_width=2,marker_size=10)
    fig.add_trace(pixel_line_profile,row=2,col=2)
    fig.add_trace(pixel_line,row=1,col=2)

    fig.update_layout(
        showlegend=False,
            template=template,
             margin=dict(r=0, t=0, b=0, l=0),
            font=dict(
            family="Latin Modern Roman,serif",
            color='white',
            size=13,
        ),   
            plot_bgcolor='rgba(0,0,0,0)',
        xaxis2 = dict(title='x (nm)',tickvals=xt,ticktext=xtl,range=[xi,xf]),
        yaxis1 = dict(title=' ',tickvals=[],range=[yi,yf]),
         scene=dict(
            xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xtl,tickvals=xt,tickprefix= "nm",),
            yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=ytl,tickvals=yt,tickprefix= "nm",),
            zaxis=dict(),
            aspectratio= {"x": 1, "y": 1, "z": 1.5},
            camera_eye= {"x": 2, "y": 2, "z": 2},
    ),
    )
    return fig


fig_3d_2d_layout(afm,'plotly_dark',attrs,10, color='Plotly3_r',tick_step=5)


16
